In [492]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))
import talib
import sqlite3 as sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.font_manager as fm
from mplfinance.original_flavor import candlestick_ohlc
from util.util import get_ucodes

In [493]:
path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
print(path_sqlite)
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()
conn.row_factory = lambda cursor, row: row[0]
no_days = 167
no_days = 350

D:\PycharmProjects\scrapy-001\data\aastock\sqlite\hk-marketwatch.db


In [494]:
data1 = {}
ucodes = ['hsi', 'hsce', 'hstech', 'ssec', 'csi300', 'sp500', 'dji', 'ixic']
ucodes = ['02800', '03033', '02822', '03188']
#ucodes = ['09988', '01810', '03690', '00700']
for ucode in ucodes:
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC LIMIT {}) AS t 
                            ORDER BY t.stime """.format(ucode, no_days)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data1[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data1[ucode].index = pd.to_datetime(data1[ucode].sdate)
    data1[ucode]['weekday'] = data1[ucode].index.dayofweek
    data1[ucode]['pchng'] = data1[ucode]['last'].pct_change()*100
conn.close()

In [495]:
no_spread = 30
data2 = {}
for ucode, df in data1.items():
    print('{}  {} ~ {}'.format(ucode, df.iloc[0].sdate, df.iloc[-1].sdate))
    data2[ucode] = pd.DataFrame(columns=['bottom', 'ceil', 'count', 'sdate', 'vol']) 
    max = df['last'].max()
    min = df['last'].min()
    spread = (max-min)/no_spread
    for i in range(0, no_spread):
        bottom = round(min + (spread*i), 2)
        ceil = round(min + (spread*(i+1)), 2)
        df2 = df.loc[(df['last'] >= bottom) & (df['last'] <= ceil)]
        count = df2.shape[0]
        sdate2 = df2['sdate'].values.tolist()
        vol = round(df.loc[(df['last'] >= bottom) & (df['last'] <= ceil)]['vol'].sum()/100000000, 3)
        data2[ucode].loc[len(data2[ucode])] = [bottom, ceil, count, ','.join(sdate2), vol]
print('-------------------------------------------------')

for ucode2, df2 in data2.items():
    row2 = data1[ucode2].iloc[-1]
    print(ucode2, row2['nmll'], row2['last'])
    df2["count"] = pd.to_numeric(df2["count"])
    n_larger = df2['vol'].nlargest(6).values.tolist()[::-1]
    spread = df2.iloc[0]['ceil'] - df2.iloc[0]['bottom']
    cum_vol = 0
    print('{}%  {}   {}'.format("{:.2f}".format(spread*10/row2['last']*100), 'aggr', 'nega'))
    for k, v in df2.iterrows():
        # 6大重倉標注
        remark = ''
        if v['vol'] in n_larger:
            remark = '<<'
            for i in range(0, n_larger.index(v['vol'])):
                remark += '--'
        # 現價
        remark2 = ''
        if row2['last'] >= v['bottom'] and row2['last'] < v['ceil']:
            remark2 = '现价'
        # 深度
        spread = v['ceil'] - v['bottom']
        p_bottom = spread/v['bottom']*100
        p_ceil = spread/v['ceil']*100
        # 分隔線
        if k%10 == 0 and k > 1:
            print('-------------------------------------------------')
        # 量
        cum_vol += v['vol']
        remark3 = ''
        if k == 9 or k == 19 or k == 29:
            remark3 = '('+str(round(cum_vol, 2))+'亿笔)'
            cum_vol = 0
        #
        print('{}   | {}% ~ {}% | {} ~ {} | {}亿笔 | {}  {} {} {}'.format(str(k+1).zfill(2), "{:.2f}".format(p_bottom), "{:.2f}".format(p_ceil), "{:.2f}".format(v['bottom']), "{:.2f}".format(v['ceil']), "{:.3f}".format(v['vol']), v['count'], remark, remark2, remark3))
    print('------------------------------------------------------------------')

02800  2019-12-03 ~ 2021-05-05
03033  2020-08-28 ~ 2021-05-05
02822  2019-12-03 ~ 2021-05-05
03188  2019-12-03 ~ 2021-05-05
-------------------------------------------------
02800 盈富基金 28.64
10.82%  aggr   nega
01   | 1.41% ~ 1.39% | 22.00 ~ 22.31 | 1.895亿笔 | 2    
02   | 1.39% ~ 1.37% | 22.31 ~ 22.62 | 0.000亿笔 | 0    
03   | 1.33% ~ 1.31% | 22.62 ~ 22.92 | 1.042亿笔 | 1    
04   | 1.35% ~ 1.33% | 22.92 ~ 23.23 | 5.677亿笔 | 5    
05   | 1.33% ~ 1.32% | 23.23 ~ 23.54 | 6.943亿笔 | 7    
06   | 1.32% ~ 1.30% | 23.54 ~ 23.85 | 6.636亿笔 | 7    
07   | 1.30% ~ 1.28% | 23.85 ~ 24.16 | 11.124亿笔 | 14    
08   | 1.24% ~ 1.23% | 24.16 ~ 24.46 | 11.896亿笔 | 15    
09   | 1.27% ~ 1.25% | 24.46 ~ 24.77 | 16.727亿笔 | 20    
10   | 1.25% ~ 1.24% | 24.77 ~ 25.08 | 9.029亿笔 | 15    (70.97亿笔)
-------------------------------------------------
11   | 1.24% ~ 1.22% | 25.08 ~ 25.39 | 14.224亿笔 | 22    
12   | 1.22% ~ 1.21% | 25.39 ~ 25.70 | 17.366亿笔 | 27  <<  
13   | 1.17% ~ 1.15% | 25.70 ~ 26.00 | 12.923亿笔 | 21    
